In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define LSTM Encoder
class LSTMEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(LSTMEncoder, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)

    def forward(self, x):
        outputs, (hidden, cell) = self.lstm(x)
        return outputs, hidden, cell

# Define LSTM Decoder
class LSTMDecoder(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers, dropout):
        super(LSTMDecoder, self).__init__()
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, hidden, cell, seq_len):
        batch_size = hidden.size(1)
        output_size = self.fc.out_features

        outputs = torch.zeros(batch_size, seq_len, output_size).to(hidden.device)
        input = encoder_outputs[:, -1, :].unsqueeze(1)  # First input to the decoder is the last hidden state of the encoder

        for t in range(seq_len):
            output, (hidden, cell) = self.lstm(input, (hidden, cell))
            output = self.fc(output.squeeze(1))
            outputs[:, t, :] = output
            input = output.unsqueeze(1)

        return outputs

# Define the Hybrid Model
class HybridModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout):
        super(HybridModel, self).__init__()
        self.encoder = LSTMEncoder(input_size, hidden_size, num_layers, dropout)
        self.decoder = LSTMDecoder(hidden_size, output_size, num_layers, dropout)

    def forward(self, past_data, seq_len):
        encoder_outputs, hidden, cell = self.encoder(past_data)
        prediction = self.decoder(encoder_outputs, hidden, cell, seq_len)
        return prediction

# Example usage
input_size = 10
hidden_size = 128
output_size = 1
num_layers = 2
dropout = 0.5
batch_size = 32
seq_len = 20

model = HybridModel(input_size, hidden_size, output_size, num_layers, dropout)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Dummy data
past_data = torch.randn(batch_size, seq_len, input_size)
targets = torch.randn(batch_size, seq_len, output_size)

# Forward pass
outputs = model(past_data, seq_len)
loss = criterion(outputs, targets)
loss.backward()
optimizer.step()

print("Output shape:", outputs.shape)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define Attention Mechanism
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))
        self.init_weights()

    def init_weights(self):
        nn.init.xavier_uniform_(self.attn.weight)
        nn.init.constant_(self.attn.bias, 0)
        nn.init.uniform_(self.v, -0.1, 0.1)

    def forward(self, hidden, encoder_outputs):
        seq_len = encoder_outputs.size(1)
        hidden = hidden.unsqueeze(1).repeat(1, seq_len, 1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        energy = energy.transpose(1, 2)
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)
        attention_weights = torch.bmm(v, energy).squeeze(1)
        return torch.softmax(attention_weights, dim=1)

# Define LSTM Encoder
class LSTMEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(LSTMEncoder, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)

    def forward(self, x):
        outputs, (hidden, cell) = self.lstm(x)
        return outputs, hidden, cell

# Define LSTM Decoder with Attention
class LSTMDecoderWithAttention(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers, dropout):
        super(LSTMDecoderWithAttention, self).__init__()
        self.attention = Attention(hidden_size)
        self.lstm = nn.LSTM(hidden_size * 2, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, hidden, cell, seq_len):
        batch_size = hidden.size(1)
        output_size = self.fc.out_features

        outputs = torch.zeros(batch_size, seq_len, output_size).to(hidden.device)
        input = encoder_outputs[:, -1, :].unsqueeze(1)  # First input to the decoder is the last hidden state of the encoder

        for t in range(seq_len):
            attn_weights = self.attention(hidden[-1], encoder_outputs)
            context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).squeeze(1)
            lstm_input = torch.cat([context, input.squeeze(1)], dim=1).unsqueeze(1)  # Add sequence dimension
            output, (hidden, cell) = self.lstm(lstm_input, (hidden, cell))
            output = self.fc(output.squeeze(1))
            outputs[:, t, :] = output
            input = output.unsqueeze(1)

        return outputs, attn_weights

# Define the Hybrid Model
class HybridModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout):
        super(HybridModel, self).__init__()
        self.encoder = LSTMEncoder(input_size, hidden_size, num_layers, dropout)
        self.decoder = LSTMDecoderWithAttention(hidden_size, output_size, num_layers, dropout)

    def forward(self, past_data, seq_len):
        encoder_outputs, hidden, cell = self.encoder(past_data)
        prediction, attn_weights = self.decoder(encoder_outputs, hidden, cell, seq_len)
        return prediction, attn_weights

# Example usage
input_size = 10
hidden_size = 128
output_size = 1
num_layers = 2
dropout = 0.5
batch_size = 32
seq_len = 20

model = HybridModel(input_size, hidden_size, output_size, num_layers, dropout)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Dummy data
past_data = torch.randn(batch_size, seq_len, input_size)
targets = torch.randn(batch_size, seq_len, output_size)

# Forward pass
outputs, attn_weights = model(past_data, seq_len)
loss = criterion(outputs, targets)
loss.backward()
optimizer.step()

print("Output shape:", outputs.shape)
print("Attention weights shape:", attn_weights.shape)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define Attention Mechanism
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))
        self.init_weights()

    def init_weights(self):
        nn.init.xavier_uniform_(self.attn.weight)
        nn.init.constant_(self.attn.bias, 0)
        nn.init.uniform_(self.v, -0.1, 0.1)

    def forward(self, hidden, encoder_outputs):
        seq_len = encoder_outputs.size(1)
        hidden = hidden.unsqueeze(1).repeat(1, seq_len, 1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        energy = energy.transpose(1, 2)
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)
        attention_weights = torch.bmm(v, energy).squeeze(1)
        return torch.softmax(attention_weights, dim=1)

# Define LSTM Encoder
class LSTMEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(LSTMEncoder, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)

    def forward(self, x):
        outputs, (hidden, cell) = self.lstm(x)
        return outputs, hidden, cell

# Define LSTM Decoder with Attention and Teacher Forcing
class LSTMDecoderWithAttention(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers, dropout):
        super(LSTMDecoderWithAttention, self).__init__()
        self.attention = Attention(hidden_size)
        self.lstm = nn.LSTM(hidden_size * 2, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, hidden, cell, targets=None, teacher_forcing_ratio=0.5):
        batch_size = hidden.size(1)
        seq_len = targets.size(1) if targets is not None else encoder_outputs.size(1)
        output_size = self.fc.out_features

        outputs = torch.zeros(batch_size, seq_len, output_size).to(hidden.device)
        input = encoder_outputs[:, -1, :].unsqueeze(1)  # First input to the decoder is the last hidden state of the encoder

        for t in range(seq_len):
            attn_weights = self.attention(hidden[-1], encoder_outputs)
            context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).squeeze(1)
            lstm_input = torch.cat([context, input.squeeze(1)], dim=1).unsqueeze(1)  # Add sequence dimension
            output, (hidden, cell) = self.lstm(lstm_input, (hidden, cell))
            output = self.fc(output.squeeze(1))
            outputs[:, t, :] = output

            teacher_force = targets is not None and torch.rand(1).item() < teacher_forcing_ratio
            input = targets[:, t].unsqueeze(1) if teacher_force else output.unsqueeze(1)

        return outputs, attn_weights

# Define the Hybrid Model
class HybridModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout):
        super(HybridModel, self).__init__()
        self.encoder = LSTMEncoder(input_size, hidden_size, num_layers, dropout)
        self.decoder = LSTMDecoderWithAttention(hidden_size, output_size, num_layers, dropout)

    def forward(self, past_data, targets=None, teacher_forcing_ratio=0.5):
        encoder_outputs, hidden, cell = self.encoder(past_data)
        prediction, attn_weights = self.decoder(encoder_outputs, hidden, cell, targets, teacher_forcing_ratio)
        return prediction, attn_weights

# Example usage
input_size = 10
hidden_size = 128
output_size = 1
num_layers = 2
dropout = 0.5
batch_size = 32
seq_len = 20

model = HybridModel(input_size, hidden_size, output_size, num_layers, dropout)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Dummy data
past_data = torch.randn(batch_size, seq_len, input_size)
targets = torch.randn(batch_size, seq_len, output_size)

# Forward pass
outputs, attn_weights = model(past_data, targets, teacher_forcing_ratio=0.5)
loss = criterion(outputs, targets)
loss.backward()
optimizer.step()

print("Output shape:", outputs.shape)
print("Attention weights shape:", attn_weights.shape)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define Attention Mechanism
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))
        self.init_weights()

    def init_weights(self):
        nn.init.xavier_uniform_(self.attn.weight)
        nn.init.constant_(self.attn.bias, 0)
        nn.init.uniform_(self.v, -0.1, 0.1)

    def forward(self, hidden, encoder_outputs):
        seq_len = encoder_outputs.size(1)
        hidden = hidden.unsqueeze(1).repeat(1, seq_len, 1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        energy = energy.transpose(1, 2)
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)
        attention_weights = torch.bmm(v, energy).squeeze(1)
        return torch.softmax(attention_weights, dim=1)

# Define LSTM Encoder
class LSTMEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(LSTMEncoder, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)

    def forward(self, x):
        outputs, (hidden, cell) = self.lstm(x)
        return outputs, hidden, cell

# Define LSTM Decoder with Attention and Teacher Forcing
class LSTMDecoderWithAttention(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers, dropout, macro_size):
        super(LSTMDecoderWithAttention, self).__init__()
        self.attention = Attention(hidden_size)
        self.lstm = nn.LSTM(hidden_size * 2 + macro_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, hidden, cell, macro_data, targets=None, teacher_forcing_ratio=0.5):
        batch_size = hidden.size(1)
        seq_len = targets.size(1) if targets is not None else encoder_outputs.size(1)
        output_size = self.fc.out_features

        outputs = torch.zeros(batch_size, seq_len, output_size).to(hidden.device)
        input = torch.cat([encoder_outputs[:, -1, :], macro_data], dim=1).unsqueeze(1)  # First input to the decoder is the last hidden state of the encoder concatenated with macro data

        for t in range(seq_len):
            attn_weights = self.attention(hidden[-1], encoder_outputs)
            context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).squeeze(1)
            lstm_input = torch.cat([context, input.squeeze(1), macro_data], dim=1).unsqueeze(1)  # Add sequence dimension
            output, (hidden, cell) = self.lstm(lstm_input, (hidden, cell))
            output = self.fc(output.squeeze(1))
            outputs[:, t, :] = output

            teacher_force = targets is not None and torch.rand(1).item() < teacher_forcing_ratio
            input = torch.cat([targets[:, t], macro_data], dim=1).unsqueeze(1) if teacher_force else torch.cat([output, macro_data], dim=1).unsqueeze(1)

        return outputs, attn_weights

# Define the Hybrid Model
class HybridModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout, macro_size):
        super(HybridModel, self).__init__()
        self.encoder = LSTMEncoder(input_size, hidden_size, num_layers, dropout)
        self.decoder = LSTMDecoderWithAttention(hidden_size, output_size, num_layers, dropout, macro_size)

    def forward(self, past_data, macro_data, targets=None, teacher_forcing_ratio=0.5):
        encoder_outputs, hidden, cell = self.encoder(past_data)
        prediction, attn_weights = self.decoder(encoder_outputs, hidden, cell, macro_data, targets, teacher_forcing_ratio)
        return prediction, attn_weights

# Example usage
input_size = 10
hidden_size = 128
output_size = 1
num_layers = 2
dropout = 0.5
batch_size = 32
seq_len = 20
macro_size = 5  # Size of today's macro data

model = HybridModel(input_size, hidden_size, output_size, num_layers, dropout, macro_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Dummy data
past_data = torch.randn(batch_size, seq_len, input_size)
macro_data = torch.randn(batch_size, macro_size)
targets = torch.randn(batch_size, seq_len, output_size)

# Forward pass
outputs, attn_weights = model(past_data, macro_data, targets, teacher_forcing_ratio=0.5)
loss = criterion(outputs, targets)
loss.backward()
optimizer.step()

print("Output shape:", outputs.shape)
print("Attention weights shape:", attn_weights.shape)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define Convolutional Layer
class ConvLayer(nn.Module):
    def __init__(self, input_size, hidden_size, kernel_size, dropout):
        super(ConvLayer, self).__init__()
        self.conv = nn.Conv1d(input_size, hidden_size, kernel_size, padding=(kernel_size - 1) // 2)
        self.bn = nn.BatchNorm1d(hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Change shape to (batch_size, input_size, seq_len)
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = x.permute(0, 2, 1)  # Change shape back to (batch_size, seq_len, hidden_size)
        return x

# Define Attention Mechanism
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))
        self.init_weights()

    def init_weights(self):
        nn.init.xavier_uniform_(self.attn.weight)
        nn.init.constant_(self.attn.bias, 0)
        nn.init.uniform_(self.v, -0.1, 0.1)

    def forward(self, hidden, encoder_outputs):
        seq_len = encoder_outputs.size(1)
        hidden = hidden.unsqueeze(1).repeat(1, seq_len, 1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        energy = energy.transpose(1, 2)
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)
        attention_weights = torch.bmm(v, energy).squeeze(1)
        return torch.softmax(attention_weights, dim=1)

# Define LSTM Encoder
class LSTMEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(LSTMEncoder, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)

    def forward(self, x):
        outputs, (hidden, cell) = self.lstm(x)
        return outputs, hidden, cell

# Define Transformer Layer
class TransformerLayer(nn.Module):
    def __init__(self, hidden_size, num_heads, num_layers, dropout):
        super(TransformerLayer, self).__init__()
        self.transformer = nn.Transformer(hidden_size, num_heads, num_layers, dropout=dropout)

    def forward(self, x):
        x = x.permute(1, 0, 2)  # Change shape to (seq_len, batch_size, hidden_size)
        x = self.transformer(x, x)
        x = x.permute(1, 0, 2)  # Change shape back to (batch_size, seq_len, hidden_size)
        return x

# Define LSTM Decoder with Attention and Teacher Forcing
class LSTMDecoderWithAttention(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers, dropout, macro_size):
        super(LSTMDecoderWithAttention, self).__init__()
        self.attention = Attention(hidden_size)
        self.lstm = nn.LSTM(hidden_size * 2 + macro_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, hidden, cell, macro_data, targets=None, teacher_forcing_ratio=0.5):
        batch_size = encoder_outputs.size(0)
        seq_len = encoder_outputs.size(1)
        output_size = self.fc.out_features

        outputs = torch.zeros(batch_size, seq_len, output_size).to(encoder_outputs.device)
        input = encoder_outputs[:, -1, :].unsqueeze(1)  # First input to the decoder is the last hidden state of the encoder

        for t in range(seq_len):
            attn_weights = self.attention(hidden[-1], encoder_outputs)
            context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).squeeze(1)
            lstm_input = torch.cat([context, input.squeeze(1), macro_data], dim=1).unsqueeze(1)  # Add sequence dimension
            output, (hidden, cell) = self.lstm(lstm_input, (hidden, cell))
            output = self.fc(output.squeeze(1))
            outputs[:, t, :] = output

            teacher_force = targets is not None and torch.rand(1).item() < teacher_forcing_ratio
            input = targets[:, t].unsqueeze(1) if teacher_force else output.unsqueeze(1)

        return outputs, attn_weights

# Define the Hybrid Model
class HybridModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout, macro_size, kernel_size, num_heads, num_transformer_layers):
        super(HybridModel, self).__init__()
        self.conv = ConvLayer(input_size, hidden_size, kernel_size, dropout)
        self.encoder = LSTMEncoder(hidden_size, hidden_size, num_layers, dropout)
        self.decoder = LSTMDecoderWithAttention(hidden_size, output_size, num_layers, dropout, macro_size)
        self.transformer = TransformerLayer(hidden_size, num_heads, num_transformer_layers, dropout)

    def forward(self, past_data, macro_data, targets=None, teacher_forcing_ratio=0.5):
        conv_output = self.conv(past_data)
        encoder_outputs, hidden, cell = self.encoder(conv_output)
        prediction, attn_weights = self.decoder(encoder_outputs, hidden, cell, macro_data, targets, teacher_forcing_ratio)
        transformer_output = self.transformer(prediction)
        return transformer_output, attn_weights

# Example usage
input_size = 10
hidden_size = 128
output_size = 1
num_layers = 2
dropout = 0.5
batch_size = 32
seq_len = 20
macro_size = 5  # Size of today's macro data
kernel_size = 3
num_heads = 8
num_transformer_layers = 2

model = HybridModel(input_size, hidden_size, output_size, num_layers, dropout, macro_size, kernel_size, num_heads, num_transformer_layers)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Dummy data
past_data = torch.randn(batch_size, seq_len, input_size)
macro_data = torch.randn(batch_size, macro_size)
targets = torch.randn(batch_size, seq_len, output_size)

# Forward pass
outputs, attn_weights = model(past_data, macro_data, targets, teacher_forcing_ratio=0.5)
loss = criterion(outputs, targets)
loss.backward()
optimizer.step()

print("Output shape:", outputs.shape)
print("Attention weights shape:", attn_weights.shape)